In [5]:
import nltk
import spacy
import pandas as pd
from tqdm import tqdm
import os
import nltk
import numpy as np
from pattern.text.en import singularize
import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import pickle
from collections import Counter

In [6]:
with open('edges.pickle', 'rb') as file:
    edge_weight = pickle.load(file)

# Graph based analysis

In [7]:
# import networkx as nx
# from pyvis.network import Network
# import matplotlib.pyplot as plt

# G = nx.Graph()
# for item in edge_weight.most_common(200):
#     G.add_edge(item[0][0], item[0][1], weight=item[1]*100)



# g = Network(height=800, width=800, notebook=True)
# g.barnes_hut()
# g.from_nx(G)
# g.show_buttons(filter_=['physics'])
# g.show("ex.html")



In [8]:
# [edge for edge in edge_weight.most_common() if "(P)" in edge[0][0] and "(I)" in edge[0][1]]

# Inter-study comparison frameworks

In [9]:
ent_df = pd.read_csv("Litcovid_treatment_ents.csv")
temp = ent_df[((ent_df.type == "Participant") & (ent_df.text == "covid-19")) |
       ((ent_df.type == "Intervention") & (ent_df.text == "hydroxychloroquine"))]
grps = temp.groupby("pmid")
grps.describe()

Unnamed: 0                                                           \
              count           mean        std       min        25%       50%   
pmid                                                                           
31981224        2.0       1.500000   2.121320       0.0       0.75       1.5   
31996494        1.0       4.000000        NaN       4.0       4.00       4.0   
32013309        1.0       6.000000        NaN       6.0       6.00       6.0   
32026148        2.0      70.500000   0.707107      70.0      70.25      70.5   
32034659        1.0      90.000000        NaN      90.0      90.00      90.0   
32052466        1.0     158.000000        NaN     158.0     158.00     158.0   
32074550        1.0     286.000000        NaN     286.0     286.00     286.0   
32075365        4.0     321.250000   6.652067     313.0     318.25     321.5   
32079150        1.0     421.000000        NaN     421.0     421.00     421.0   
32081636        1.0     440.000000        NaN     440.0     440.00     440.0   
32096367        2.0     634.000000   2.828427     632.0     633.00     634.0   
32096395        1.0     659.000000        NaN     659.0     659.00     659.0   
32096396        1.0     661.000000        NaN     661.0     661.00     661.0   
32104907        1.0     692.000000        NaN     692.0     692.00     692.0   
32106567        1.0     786.000000        NaN     786.0     786.00     786.0   
32108862        1.0     819.000000        NaN     819.0     819.00     819.0   
32115733        2.0     852.500000   0.707107     852.0     852.25     852.5   
32117569        1.0     859.000000        NaN     859.0     859.00     859.0   
32118639        3.0     941.000000   4.000000     937.0     939.00     941.0   
32125140        1.0    1094.000000        NaN    1094.0    1094.00    1094.0   
32125362        2.0    1129.500000   4.949747    1126.0    1127.75    1129.5   
32129518        2.0    1146.000000   1.414214    1145.0    1145.50    1146.0   
32134278        1.0    1191.000000        NaN    1191.0    1191.00    1191.0   
32145714        4.0    1252.000000  17.204651    1230.0    1246.50    1253.0   
32147496        1.0    1276.000000        NaN    1276.0    1276.00    1276.0   
32147628        1.0    1279.000000        NaN    1279.0    1279.00    1279.0   
32147731        1.0    1280.000000        NaN    1280.0    1280.00    1280.0   
32149769        2.0    1315.000000   2.828427    1313.0    1314.00    1315.0   
32150618       12.0    1366.166667  10.726631    1347.0    1359.75    1369.5   
32152082        1.0    1386.000000        NaN    1386.0    1386.00    1386.0   
...             ...            ...        ...       ...        ...       ...   
32773926        1.0  120727.000000        NaN  120727.0  120727.00  120727.0   
32774170        1.0  120742.000000        NaN  120742.0  120742.00  120742.0   
32774508        4.0  120757.000000   1.825742  120755.0  120755.75  120757.0   
32774918        3.0  120773.333333   1.527525  120772.0  120772.50  120773.0   
32774998        2.0  120789.500000   2.121320  120788.0  120788.75  120789.5   
32775090        1.0  120813.000000        NaN  120813.0  120813.00  120813.0   
32775778        3.0  120844.666667   8.736895  120835.0  120841.00  120847.0   
32775814        2.0  120878.500000   6.363961  120874.0  120876.25  120878.5   
32775837        1.0  120916.000000        NaN  120916.0  120916.00  120916.0   
32775986        5.0  120939.800000   5.674504  120932.0  120936.00  120942.0   
32776022        2.0  120970.000000   1.414214  120969.0  120969.50  120970.0   
32776139        1.0  120973.000000        NaN  120973.0  120973.00  120973.0   
32776161        2.0  120999.500000   0.707107  120999.0  120999.25  120999.5   
32776197        2.0  121030.000000  11.313708  121022.0  121026.00  121030.0   
32776298        3.0  121081.666667  14.742230  121065.0  121076.00  121087.0   
32776309        1.0  121150.000000        NaN  121150.0  121150.00  121150.0   
32776354 

In [10]:
tuples = list(zip(*[ent_df['pmid'], ent_df['Unnamed: 0']]))
index = pd.MultiIndex.from_tuples(tuples, names=['pmid', 'orig_index'])

In [11]:
temp_ent_df = ent_df.copy() #multi-index version
temp_ent_df.index = index
temp_ent_df[((temp_ent_df.type == "Participant") & (temp_ent_df.text == "covid-19")) |
       ((temp_ent_df.type == "Intervention") & (temp_ent_df.text == "hydroxychloroquine"))]

Unnamed: 0                text          type  negation  \
pmid     orig_index                                                           
31981224 0                    0            covid-19   Participant         0   
         3                    3            covid-19   Participant         0   
31996494 4                    4            covid-19   Participant         0   
32013309 6                    6            covid-19   Participant         0   
32026148 70                  70            covid-19   Participant         0   
         71                  71            covid-19   Participant         0   
32034659 90                  90            covid-19   Participant         0   
32052466 158                158            covid-19   Participant         0   
32074550 286                286            covid-19   Participant         0   
32075365 313                313  hydroxychloroquine  Intervention         0   
         320                320  hydroxychloroquine  Intervention         0   
         323                323  hydroxychloroquine  Intervention         0   
         329                329  hydroxychloroquine  Intervention         0   
32079150 421                421            covid-19   Participant         0   
32081636 440                440            covid-19   Participant         0   
32096367 632                632            covid-19   Participant         0   
         636                636            covid-19   Participant         0   
32096395 659                659            covid-19   Participant         0   
32096396 661                661            covid-19   Participant         0   
32104907 692                692            covid-19   Participant         0   
32106567 786                786            covid-19   Participant         0   
32108862 819                819            covid-19   Participant         0   
32115733 852                852            covid-19   Participant         0   
         853                853            covid-19   Participant         0   
32117569 859                859            covid-19   Participant         0   
32118639 937                937            covid-19   Participant         0   
         941                941            covid-19   Participant         0   
         945                945            covid-19   Participant         0   
32125140 1094              1094            covid-19   Participant         0   
32125362 1126              1126            covid-19   Participant         0   
...                         ...                 ...           ...       ...   
32776309 121150          121150            covid-19   Participant         0   
32776354 121174          121174            covid-19   Participant         0   
32776534 121176          121176            covid-19   Participant         0   
32776551 121224          121224            covid-19   Participant         0   
32776573 121247          121247            covid-19   Participant         0   
         121249          121249            covid-19   Participant         0   
         121253          121253            covid-19   Participant         0   
         121254          121254            covid-19   Participant         0   
32776581 121298          121298            covid-19   Participant         0   
32776617 121323          121323            covid-19   Participant         0   
32776905 121336          121336            covid-19   Participant         0   
         121341          121341            covid-19   Participant         0   
32777054 121351          121351            covid-19   Participant         0   
32777263 121439          121439  hydroxychloroquine  Intervention         0   
         121440          121440            covid-19   Participant         0   
         121444          121444  hydroxychloroquine  Intervention         0   
32777300 121494          121494            covid-19   Participant         0   
         121496          121496            covid-19   Participant      

# Get pmids with entity count 
Section-agnostic

In [12]:
ind_pmid_dfs = []
grps = ent_df.groupby("pmid")
for p in tqdm(grps.groups.keys()):
    temp = grps.get_group(p)
    temp = temp.groupby(['text','type']).size().reset_index().rename(columns={0:'count'})
    temp["pmid"] = p
    ind_pmid_dfs.append(temp)

count_ent_df = pd.concat(ind_pmid_dfs)


    

100%|█████████████████████████████████████████████████████████████████████████████| 6002/6002 [00:29<00:00, 202.65it/s]


In [13]:
count_ent_df

,text,type,count,pmid
0,covid-19,Participant,2,31981224
1,host defense against,Outcome,1,31981224
2,relevant,Participant,1,31981224
0,covid-19,Participant,1,31996494
0,covid-19,Participant,1,32013309
1,inflammation,Outcome,1,32013309
2,inhibited,modifier,1,32013309
0,accelerate,modifier,1,32015560
1,control,Outcome,1,32015560
0,2019-nCoV infection,Participant,1,32017984


In [14]:
ent_df = pd.read_csv("LitCovid_treatment_ents.csv")
def get_pmids_with_entities(df, ents):
    temp_df = df.copy()
    for ent in ents:
        if type(ent[0]) == list:
            temp_df = temp_df[(temp_df.text.isin(ent[0]))&(temp_df.type == ent[1])].copy()
        else:
            temp_df = temp_df[(temp_df.text == ent[0])&(temp_df.type == ent[1])].copy()

    
    return temp_df, temp_df.pmid.unique()

temp, temp_pmids = get_pmids_with_entities(ent_df, [(["age", "aged"], "Participant")])

In [15]:
len(temp_pmids)

593

# Frequencies of PICO element text

In [16]:
tuples = list(zip(*[ent_df['pmid'], ent_df['Unnamed: 0']]))
index = pd.MultiIndex.from_tuples(tuples, names=['pmid', 'orig_index'])
pmid_ent_df = ent_df.copy()
pmid_ent_df.index = index

In [17]:
ent_df[ent_df.pmid == 32234468]

,Unnamed: 0,text,type,negation,start,sentence,subtag,section,pmid
4351,4351,mean incubation period,Outcome,0,1,"The mean incubation period is 2-14 days , and ...",None,TITLE,32234468
4352,4352,2-14 days,measure,0,5,"The mean incubation period is 2-14 days , and ...",None,TITLE,32234468
4353,4353,basic reproduction number,Outcome,0,10,"The mean incubation period is 2-14 days , and ...",None,TITLE,32234468
4354,4354,2.24-3.58,measure,0,14,"The mean incubation period is 2-14 days , and ...",None,TITLE,32234468
4355,4355,covid-19,Intervention,0,19,This paper reviews the literature on all avail...,None,BACKGROUND,32234468
4356,4356,antiviral agents,Intervention,0,3,"Treatments , including antiviral agents , chlo...",None,BACKGROUND,32234468
4357,4357,hydroxychloroquine,Intervention,0,6,"Treatments , including antiviral agents , chlo...",None,BACKGROUND,32234468
4358,4358,hydroxychloroquine,Intervention,0,8,"Treatments , including antiviral agents , chlo...",None,BACKGROUND,32234468
4359,4359,corticosteroid,Intervention,0,10,"Treatments , including antiviral agents , chlo...",None,BACKGROUND,32234468
4360,4360,antibody,Intervention,0,12,"Treatments , including antiviral agents , chlo...",None,BACKGROUND,32234468


In [18]:
hcq_pmids = ent_df[(ent_df.text == "hydroxychloroquine") & (ent_df.type == "Intervention")].pmid.unique()
hcq_ent_df = pmid_ent_df.loc[hcq_pmids]
hcq_ent_df[(hcq_ent_df.text == "diabetes") & (hcq_ent_df.type == "Participant")]

Unnamed: 0      text         type  negation  start  \
pmid     orig_index                                                       
32278764 6359              6359  diabetes  Participant         0      3   
         6374              6374  diabetes  Participant         0     10   
         6380              6380  diabetes  Participant         0      0   
32390367 21578            21578  diabetes  Participant         0     14   
32392282 22176            22176  diabetes  Participant         0     15   
32401405 23653            23653  diabetes  Participant         0      4   
32402056 23762            23762  diabetes  Participant         0     12   
32514228 47383            47383  diabetes  Participant         0      5   
32546437 54643            54643  diabetes  Participant         0     14   
32586363 64226            64226  diabetes  Participant         0     18   
32610873 70251            70251  diabetes  Participant         0     13   
32628003 74265            74265  diabetes  Participant         0      3   
32649791 80211            80211  diabetes  Participant         0      9   
32665039 85386            85386  diabetes  Participant         0     23   
32711003 99316            99316  diabetes  Participant         0      7   
         99323            99323  diabetes  Participant         0     17   
         99333            99333  diabetes  Participant         0     10   

                                                              sentence subtag  \
pmid     orig_index                                                             
32278764 6359        Older age , diabetes and other comorbidities a...   None   
         6374        Hypoglycemic events may occur with chloroquine...   None   
         6380                          diabetes , and its management .   None   
32390367 21578       Patients who received ICU care were significan...   None   
32392282 22176       azithromycin , or both were more likely than t...   None   
32401405 23653       use in patients with diabetes , even in light ...   None   
32402056 23762       Comorbid conditions included hypertension in 2...   None   
32514228 47383       Patients with liver disease , diabetes , high ...   None   
32546437 54643       Fifty one percent ( n=32 ) of patients had at ...   None   
32586363 64226       the last 2 months , having COVID-19 in the fir...   None   
32610873 70251       The most prevalent co morbidity was hypertensi...   None   
32628003 74265       Common comorbidities were diabetes mellitus ( ...   None   
32649791 80211       Prevalent comorbidities included hypertension ...   None   
32665039 85386       temporarily suspended/replaced ; known pre-exi...   None   
32711003 99316       In addition , it is suggested the diabetic pat...   None   
         99323       This paper also provides recommendations and g...   None   
         99333       In the subsequent meta-analysis 14.5 % of the ...   None   

                         section      pmid  
pmid     orig_index                         
32278764 6359         BACKGROUND  32278764  
         6374            RESULTS  32278764  
         6380            RESULTS  32278764  
32390367 21578           RESULTS  32390367  
32392282 22176           METHODS  32392282  
32401405 23653        BACKGROUND  32401405  
32402056 23762           METHODS  32402056  
32514228 47383           RESULTS  32514228  
32546437 54643           RESULTS  32546437  
32586363 64226       CONCLUSIONS  32586363  
32610873 70251           METHODS  32610873  
32628003 74265        BACKGROUND  32628003  
32649791 80211           RESULTS  32649791  
32665039 85386         OBJECTIVE  32665039  
32711003 99316        BACKGROUND  32711003  
         99323         OBJECTIVE  32711003  
         99333           METHODS  32711003

In [19]:
hcq_pmids = ent_df[(ent_df.text == "hydroxychloroquine") & (ent_df.type == "Intervention")].pmid.unique()
hcq_ent_df = pmid_ent_df.loc[hcq_pmids]
hcq_ent_df[(hcq_ent_df.text == "pregnant") & (hcq_ent_df.type == "Participant")]

,,Unnamed: 0,text,type,negation,start,sentence,subtag,section,pmid
pmid,orig_index,,,,,,,,,
32196083,2437,2437,pregnant,Participant,0,13,It has a safer clinical profile and is suitabl...,None,CONCLUSIONS,32196083
32665039,85372,85372,pregnant,Participant,0,5,Exclusion criteria include : currently pregnan...,None,OBJECTIVE,32665039


# Get PMIDs w/ certain terms for manual review

In [14]:
ent_data_df = pd.read_csv("Litcovid_treatment_ents.csv")
def get_manual_review_csvs(df, ents):
    for ent in ents:
    temp, temp_pmids = get_pmids_with_entities(df, ents)
    temp.drop(["type", "negation"], axis=1, inplace=True)
    temp["subtag"] = ""
    temp.reset_index(inplace=True)
    temp.drop(["Unnamed: 0", "section", "index"], axis=1, inplace=True)
    temp = temp[["pmid", "text", "sentence", "subtag"]]
    
    term = ents[0][0][0]
    print("There are {} PMIDs".format(temp.shape[0]))
    
    if temp.shape[0] < 30:
          jane_pmids = int(round(temp.shape[0])/3)
            
    else:
        jane_pmids = int(round(temp.shape[0])/30)
    
    temp.iloc[:jane_pmids].to_csv("Severe_PMIDs/jane_{}_pmids.csv".format(term))
    temp.iloc[jane_pmids:jane_pmids*2].to_csv("Severe_PMIDs/hao_{}_pmids.csv".format(term))
    temp.iloc[jane_pmids*2:jane_pmids*3].to_csv("Severe_PMIDs/fengyang_{}_pmids.csv".format(term))

    return temp
term_df = get_manual_review_csvs(ent_data_df, [(["age", "aged"], "Participant")])
term_df = get_manual_review_csvs(ent_data_df, [(["severe"], "Outcome")])


There are 819 PMIDs
There are 26 PMIDs


In [26]:
ent_df  = pd.read_csv("Litcovid_treatment_ents.csv")
def get_manual_review_csvs(df, ents):
    i = 0
    names = ["jane", "hao", "fy"]
    for ent in ents[0][0]:
        temp, temp_pmids = get_pmids_with_entities(df, [([ent], "Outcome")])
        temp.drop(["type", "negation"], axis=1, inplace=True)
        temp.reset_index(inplace=True)
        temp.drop(["Unnamed: 0", "section", "index"], axis=1, inplace=True)
        temp = temp[["pmid", "text", "sentence", "subtag"]]

        term = ent
        print("There are {} PMIDs".format(temp.shape[0]))
        temp.iloc[:30].to_csv("Severe_PMIDs/{}_{}_pmids.csv".format(names[i], term))
        i+=1
    return temp
# term_df = get_manual_review_csvs(ent_data_df, [(["age", "aged"], "Participant")])
term_df = get_manual_review_csvs(ent_df, [(["mechanical ventilation", "in-hospital mortality", "severe"], "Outcome")])

There are 114 PMIDs


FileNotFoundError: [Errno 2] No such file or directory: 'Severe_PMIDs/jane_mechanical ventilation_pmids.csv'

In [27]:
ent_df  = pd.read_csv("Litcovid_treatment_ents.csv")
temp, temp_pmids = get_pmids_with_entities(ent_df, [(["tracheostomy"], "Outcome")])
temp.drop(["type", "negation"], axis=1, inplace=True)
temp.reset_index(inplace=True)
temp.drop(["Unnamed: 0", "section", "index"], axis=1, inplace=True)
temp = temp[["pmid", "text", "sentence", "subtag"]]

print("There are {} PMIDs".format(temp.shape[0]))
temp.iloc[:30].to_csv("Manual_Review_PMIDs/{}_{}_pmids.csv".format("marguerite", "tracheostomy"))

There are 7 PMIDs


In [33]:
ent_df[ent_df.text == "tracheostomy"].to_csv("Manual_Review_PMIDs/{}_{}_pmids.csv".format("marguerite", "tracheostomy"))

In [34]:
ent_df[ent_df.text == "tracheostomy"].pmid.unique()

array([32427403, 32450761, 32458121, 32506145, 32600107, 32656673,
       32719738, 32740720, 32741194, 32757019, 32758308], dtype=int64)